In [33]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the data
data = pd.read_csv('combined_data.csv', delimiter=';')

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for column in data.columns[:-1]:  # Exclude the 'Label' column
    data[column] = le.fit_transform(data[column].astype(str))

# Encode the labels as well
data['Label'] = le.fit_transform(data['Label'].astype(str))

# Split the data into features (X) and target (y)
X = data.iloc[:, :-1].values  # All columns except the last one
y = data['Label'].values      # The last column is the target

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

# Save the model in TensorFlow SavedModel format
model.export('tensorflow_model')
model.save('tensorflow_model.h5')


Epoch 1/10


/Users/berkebozaci/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2589 - loss: 2.7603 - val_accuracy: 0.7520 - val_loss: 1.2224
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.7711 - loss: 0.9825 - val_accuracy: 0.8610 - val_loss: 0.6177
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.8667 - loss: 0.5137 - val_accuracy: 0.8170 - val_loss: 0.4494
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.8858 - loss: 0.3671 - val_accuracy: 0.8950 - val_loss: 0.3445
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.9117 - loss: 0.2764 - val_accuracy: 0.9160 - val_loss: 0.2742
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.9293 - loss: 0.2377 - val_accuracy: 0.9230 - val_loss: 0.2478
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - accuracy: 0.9286 - loss: 0.2183 - val_accuracy: 0.9170 - val_loss: 0.2442
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.9371 - loss: 0.1989 - val_accuracy:

INFO:tensorflow:Assets written to: tensorflow_model/assets


Saved artifact at 'tensorflow_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor_78')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  5530343696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5547213648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5547213456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5547214416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5547214224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5547215184: TensorSpec(shape=(), dtype=tf.resource, name=None)


enter the command to convert `tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve tensorflow_model/ model_tfjs/`


In [34]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the model from the .h5 file
model = tf.keras.models.load_model('tensorflow_model.h5')

# Load the dataset to fit the LabelEncoder (used during training)
data = pd.read_csv('combined_data.csv', delimiter=';')
le = LabelEncoder()
le.fit(data['Label'])


LabelEncoder()

In [44]:

# Create mock data - a single example with the same shape as the training data
mock_data = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1, -1) 
#[0, 2, 2, 0, 2, 0, 2, 0, 2, 0]
#[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#[0, 4, 1, 2, 0, 2, 0, 2, 0, 2]
#[1, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> B
# [0, 2, 1, 2, 1, 2, 1, 2, 1, 2] -> O
# [0, 4, 1, 2, 1, 2, 1, 2, 1, 2] -> C
# [0, 0, 2, 2, 2, 2, 2, 0, 2, 0] -> N

# Predict the class for the mock data
prediction = model.predict(mock_data)

print(prediction)

# Get the predicted class index
predicted_class = np.argmax(prediction, axis=-1)

# Decode the predicted class index to the ASL letter
predicted_label = le.inverse_transform(predicted_class)

print(f'Predicted class index: {predicted_class[0]}')
print(f'Predicted ASL letter: {predicted_label[0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[8.3170342e-01 7.1852305e-08 4.4895601e-06 3.5948553e-06 5.6419147e-08
  2.2289931e-04 1.3121569e-03 1.9273166e-08 1.2221383e-05 2.8064712e-06
  1.4864708e-03 3.8049513e-04 3.1709176e-02 1.1619674e-01 1.9897989e-05
  1.0685655e-02 3.6448539e-05 1.5683499e-05 1.2704138e-04 8.6223215e-09
  5.6866837e-05 8.6635879e-08 1.3803915e-05 6.0097785e-03]]
Predicted class index: 0
Predicted ASL letter: A


In [45]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the original CSV data
data = pd.read_csv('combined_data.csv', delimiter=';')

# Initialize LabelEncoder for each feature column
encoders = {col: LabelEncoder() for col in data.columns[:-1]}  # Exclude the Label column

# Encode each feature column
for col, encoder in encoders.items():
    data[col] = encoder.fit_transform(data[col].astype(str))

# Encode the Label column
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'].astype(str))

# Save the formatted data to a new CSV file
data.to_csv('formatted_data.csv', index=False)

# Display the first few rows of the new formatted data
print(data.head())


   Thumb Curl  Thumb Direction  Index Curl  Index Direction  Middle Curl  \
0           0                2           0                0            0   
1           0                2           0                0            0   
2           0                2           0                0            0   
3           0                2           0                0            0   
4           0                2           0                0            0   

   Middle Direction  Ring Curl  Ring Direction  Pinky Curl  Pinky Direction  \
0                 4          0               4           2                4   
1                 4          0               4           2                4   
2                 4          0               4           2                4   
3                 4          0               4           2                4   
4                 4          0               4           2                4   

   Label  
0      8  
1      8  
2      8  
3      8  
4      8  


In [51]:
# Create mock data - a single example with the same shape as the training data
mock_data = np.array([1,1,0,3,0,4,0,1,2,1]).reshape(1, -1) 
# 1,2,0,0,0,1,0,4,0,4 -> M (11)
# 1,1,2,3,0,4,0,4,0,4 -> L (10)
# 1,1,0,3,0,4,0,1,2,1 -> Y (23)
# 1,2,0,0,0,1,0,0,0,4 -> N (12)
# 1,1,0,0,0,1,1,0,1,0 -> O (13)
# 1,4,2,1,0,2,0,3,0,3 -> X (22)
# 0,2,0,0,0,4,0,4,2,4 -> I (8)
# 1,4,2,2,2,3,0,3,0,3 -> H (7)
# 1,5,2,3,1,4,0,4,0,4 -> K (9)
# 0,2,2,3,2,4,2,4,1,4 -> W (21)
# 1,1,0,0,0,4,0,4,0,4 -> A (0)
# 1,2,2,3,2,4,0,0,0,4 -> V (20)
# 1,5,0,0,0,4,0,4,0,4 -> T (18)
# 1,5,2,3,2,4,2,4,2,4 -> B (1)
# 1,3,1,0,1,1,1,0,1,0 -> C (2)
# 0,2,2,3,2,4,0,4,0,4 -> U (19)
# 1,1,1,0,2,4,2,4,2,4 -> F (5)
# 1,4,1,2,0,2,0,3,0,3 -> Q (15)
# 1,2,2,1,1,3,0,3,0,3 -> P (14)
# 1,4,2,1,0,2,0,1,0,1 -> G (6)
# 0,2,0,3,0,4,0,4,0,1 -> E (4)
# 1,2,2,3,2,4,0,4,0,0 -> R (16)
# 0,5,0,3,0,4,0,4,0,1 -> S (17)
# 0,5,2,3,1,4,1,4,1,1 -> D (3)



# Predict the class for the mock data
prediction = model.predict(mock_data)

print(prediction)

# Get the predicted class index
predicted_class = np.argmax(prediction, axis=-1)

# Decode the predicted class index to the ASL letter
predicted_label = le.inverse_transform(predicted_class)

print(f'Predicted class index: {predicted_class[0]}')
print(f'Predicted ASL letter: {predicted_label[0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[[1.8616515e-04 2.3027675e-11 3.0876191e-16 1.3574682e-06 1.9679287e-07
  5.5809642e-06 2.4562557e-13 1.3291435e-17 1.2355058e-04 4.7250784e-12
  1.5306637e-06 5.9875510e-10 9.7410020e-07 9.6009270e-04 9.2266570e-15
  3.6370853e-08 3.0526710e-09 3.1786275e-08 4.6510264e-11 5.6436989e-13
  8.1461987e-10 8.9226537e-13 3.9418463e-13 9.9872041e-01]]
Predicted class index: 23
Predicted ASL letter: Y


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the model from the .h5 file
model = tf.keras.models.load_model('tensorflow_model.h5')

# Load the dataset to fit the LabelEncoder (used during training)
data = pd.read_csv('combined_data.csv', delimiter=';')
le = LabelEncoder()
le.fit(data['Label'])

# Define a list of mock data inputs and their expected ASL letters
mock_data_list = [
    (np.array([1, 2, 0, 0, 0, 1, 0, 4, 0, 4]), "M"),  # M (11)
    (np.array([1, 1, 2, 3, 0, 4, 0, 4, 0, 4]), "L"),  # L (10)
    (np.array([1, 1, 0, 3, 0, 4, 0, 1, 2, 1]), "Y"),  # Y (23)
    (np.array([1, 2, 0, 0, 0, 1, 0, 0, 0, 4]), "N"),  # N (12)
    (np.array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0]), "O"),  # O (13)
    (np.array([1, 4, 2, 1, 0, 2, 0, 3, 0, 3]), "X"),  # X (22)
    (np.array([0, 2, 0, 0, 0, 4, 0, 4, 2, 4]), "I"),  # I (8)
    (np.array([1, 4, 2, 2, 2, 3, 0, 3, 0, 3]), "H"),  # H (7)
    (np.array([1, 5, 2, 3, 1, 4, 0, 4, 0, 4]), "K"),  # K (9)
    (np.array([0, 2, 2, 3, 2, 4, 2, 4, 1, 4]), "W"),  # W (21)
    (np.array([1, 1, 0, 0, 0, 4, 0, 4, 0, 4]), "A"),  # A (0)
    (np.array([1, 2, 2, 3, 2, 4, 0, 0, 0, 4]), "V"),  # V (20)
    (np.array([1, 5, 0, 0, 0, 4, 0, 4, 0, 4]), "T"),  # T (18)
    (np.array([1, 5, 2, 3, 2, 4, 2, 4, 2, 4]), "B"),  # B (1)
    (np.array([1, 3, 1, 0, 1, 1, 1, 0, 1, 0]), "C"),  # C (2)
    (np.array([0, 2, 2, 3, 2, 4, 0, 4, 0, 4]), "U"),  # U (19)
    (np.array([1, 1, 1, 0, 2, 4, 2, 4, 2, 4]), "F"),  # F (5)
    (np.array([1, 4, 1, 2, 0, 2, 0, 3, 0, 3]), "Q"),  # Q (15)
    (np.array([1, 2, 2, 1, 1, 3, 0, 3, 0, 3]), "P"),  # P (14)
    (np.array([1, 4, 2, 1, 0, 2, 0, 1, 0, 1]), "G"),  # G (6)
    (np.array([0, 2, 0, 3, 0, 4, 0, 4, 0, 1]), "E"),  # E (4)
    (np.array([1, 2, 2, 3, 2, 4, 0, 4, 0, 0]), "R"),  # R (16)
    (np.array([0, 5, 0, 3, 0, 4, 0, 4, 0, 1]), "S"),  # S (17)
    (np.array([0, 5, 2, 3, 1, 4, 1, 4, 1, 1]), "D")  # D (3)
]
i = 0
# Loop through each mock data and print the predicted letter
for mock_data, expected_label in mock_data_list:
    # Reshape the data to match the input shape of the model
    print(i)
    mock_data = mock_data.reshape(1, -1)

    # Predict the class for the mock data
    prediction = model.predict(mock_data)

    # Get the predicted class index
    predicted_class = np.argmax(prediction, axis=-1)

    # Decode the predicted class index to the ASL letter
    predicted_label = le.inverse_transform(predicted_class)

    # Print the result
    print(f'Input data: {mock_data.flatten()}')
    print(f'Predicted class index: {predicted_class[0]}')
    print(f'Predicted ASL letter: {predicted_label[0]}')
    print(f'Expected ASL letter: {expected_label}')
    print('---')
    i += 1


0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Input data: [1 2 0 0 0 1 0 4 0 4]
Predicted class index: 12
Predicted ASL letter: N
Expected ASL letter: M
---
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Input data: [1 1 2 3 0 4 0 4 0 4]
Predicted class index: 10
Predicted ASL letter: L
Expected ASL letter: L
---
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Input data: [1 1 0 3 0 4 0 1 2 1]
Predicted class index: 23
Predicted ASL letter: Y
Expected ASL letter: Y
---
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Input data: [1 2 0 0 0 1 0 0 0 4]
Predicted class index: 12
Predicted ASL letter: N
Expected ASL letter: N
---
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Input data: [1 1 0 0 0 1 1 0 1 0]
Predicted class index: 13
Predicted ASL letter: O
Expected ASL letter: O
---
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Input data: [1 4 2 1 0 2 0 3 0 3]
Predicted class index: 22
Predicted ASL letter: X
Expected ASL letter: X
---
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Input data: [0 2 0 0 0 4 0 4 2 4]
Predicted class index:

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the original CSV data
data = pd.read_csv('combined_data.csv', delimiter=';')

# Initialize LabelEncoder for each feature column
encoders = {col: LabelEncoder() for col in data.columns[:-1]}  # Exclude the Label column

# Encode each feature column and store the mappings
mappings = {}
for col, encoder in encoders.items():
    data[col] = encoder.fit_transform(data[col].astype(str))
    mappings[col] = {label: index for index, label in enumerate(encoder.classes_)}

# Print the mappings for each feature
for col, mapping in mappings.items():
    print(f"Mapping for {col}: {mapping}")

# Encode the Label column
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'].astype(str))

# Save the formatted data to a new CSV file
data.to_csv('formatted_data_map.csv', index=False)

# Display the first few rows of the new formatted data
print(data.head())


Mapping for Thumb Curl: {'Half Curl': 0, 'No Curl': 1}
Mapping for Thumb Direction: {'Diagonal Down Right': 0, 'Diagonal Up Left': 1, 'Diagonal Up Right': 2, 'Horizontal Left': 3, 'Horizontal Right': 4, 'Vertical Up': 5}
Mapping for Index Curl: {'Full Curl': 0, 'Half Curl': 1, 'No Curl': 2}
Mapping for Index Direction: {'Diagonal Up Left': 0, 'Diagonal Up Right': 1, 'Horizontal Right': 2, 'Vertical Up': 3}
Mapping for Middle Curl: {'Full Curl': 0, 'Half Curl': 1, 'No Curl': 2}
Mapping for Middle Direction: {'Diagonal Down Right': 0, 'Diagonal Up Left': 1, 'Diagonal Up Right': 2, 'Horizontal Right': 3, 'Vertical Up': 4}
Mapping for Ring Curl: {'Full Curl': 0, 'Half Curl': 1, 'No Curl': 2}
Mapping for Ring Direction: {'Diagonal Up Left': 0, 'Diagonal Up Right': 1, 'Horizontal Left': 2, 'Horizontal Right': 3, 'Vertical Up': 4}
Mapping for Pinky Curl: {'Full Curl': 0, 'Half Curl': 1, 'No Curl': 2}
Mapping for Pinky Direction: {'Diagonal Up Left': 0, 'Diagonal Up Right': 1, 'Horizontal Left